In [5]:
import numpy as np
from PIL import Image

In [6]:
class LSB():
    def __init__(self,image, message):
        self.message = message
        self.image = image
        self.binary = ""

    def text_in_binary(self):
        for i in range(len(self.message)):
            binär = format(ord(self.message[i]),"08b")
            self.binary += binär
        self.message = self.binary
        return self.message

    def pictur_in_binary(self):
        yourImage = np.vectorize(lambda x: format(x, '08b'))(self.image)
        self.image = yourImage
        return self.image
        
    def message_in_pictur(self):
        shape = self.image.shape
        count = 0
        for row in range(shape[0]):
            for elem in range(shape[1]):
                for col in range(3):
                    mesimag = self.image[row,elem,col]
                    mesimage = mesimag[:7]
                    if count < len(self.message):
                        mesimage += self.message[count]
                    else:
                        mesimage += "0"
                    count += 1
                    self.image[row][elem][col] = mesimage
        return self.image

    def binary_in_pictur(self):
        yourImage = np.vectorize(lambda x: int(x,2))(self.image)
        self.image = yourImage
        return self.image

    def message_out_image(self):
        shape = self.image.shape
        bintext = ""
        count = 0
        max_bits = 15 * 8  # 15 Zeichen = 120 Bits

        for row in range(shape[0]):
            for elem in range(shape[1]):
                for col in range(3):
                    pixel = self.image[row, elem, col]
                    bintext += pixel[7]
                    count += 1
                    if count >= max_bits:
                        self.message = bintext
                        return self.message

        self.message = bintext  # falls Bild zu klein
        return self.message


    def bin_to_string(self):
        self.message = self.message[:120]  # Sicherstellen, dass es max. 15 Zeichen sind
        chars = [self.message[i:i+8] for i in range(0, len(self.message), 8)]
        self.message = ''.join(chr(int(b, 2)) for b in chars)
        self.message = self.message.rstrip('\x00')
        return self.message

        
    def encode(self):
        self.text_in_binary()
        self.pictur_in_binary()
        self.message_in_pictur()
        return self.binary_in_pictur()

    def decode(self):
        self.pictur_in_binary()
        self.message_out_image()
        return self.bin_to_string()

In [7]:
class Alpha_Channel():
    def __init__(self,image, message):
        self.message = message
        self.image = image
        self.binary = ""

    def image_alpha(self):
        
        self.image = self.image.astype(np.uint8)
        img = Image.fromarray(self.image)
        img = img.convert("RGBA")
        datum = img.getdata()

        newData = []
        for item in datum:
            if item[0] == 255 and item [1] == 255 and item[2] == 255:
                newData.append((255,255,255, 0))
            else:
                newData.append(item)
        img.putdata(newData)
        self.image = np.array(img)
        return self.image

    def text_in_binary(self):
        for i in range(len(self.message)):
            binär = format(ord(self.message[i]),"08b")
            self.binary += binär
        self.message = self.binary
        return self.message

    def pictur_in_binary(self):
        yourImage = np.vectorize(lambda x: format(x, '08b'))(self.image)
        self.image = yourImage
        return self.image
        
    def message_in_pictur(self):
        shape = self.image.shape
        count = 0
        for row in range(shape[0]):
            for elem in range(shape[1]):
                mesimag = self.image[row,elem,3]
                mesimage = mesimag[:7]
                if count < len(self.message):
                    mesimage += self.message[count]
                else:
                    mesimage += "0"
                count += 1
                self.image[row][elem][3] = mesimage
                
        return self.image

    def binary_in_pictur(self):
        yourImage = np.vectorize(lambda x: int(x,2))(self.image)
        self.image = yourImage
        return self.image


    def message_out_image(self):
        shape = self.image.shape
        bintext = ""
        count = 0
        max_bits = 25 * 8  

        for row in range(shape[0]):
            for elem in range(shape[1]):
                pixel = self.image[row][elem][3]
                bintext += pixel[7]
                count += 1
                if count >= max_bits:
                    self.message = bintext
                    return self.message

        self.message = bintext  # falls Bild kleiner ist als erwartet
        return self.message


    def bin_to_string(self):
        self.message = self.message[:200]
        chars = [self.message[i:i+8] for i in range(0, len(self.message), 8)]
        self.message = ''.join(chr(int(b, 2)) for b in chars)
        self.message = self.message.rstrip('\x00')
        return self.message

        
    def encode(self):
        self.image_alpha()
        self.text_in_binary()
        self.pictur_in_binary()
        self.message_in_pictur()
        return self.binary_in_pictur()

    def decode(self):
        self.pictur_in_binary()
        self.message_out_image()
        return self.bin_to_string()
